In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/14 00:08:58 WARN Utils: Your hostname, NivethanAsus resolves to a loopback address: 127.0.1.1; using 172.20.150.32 instead (on interface eth0)
23/09/14 00:08:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/14 00:09:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/14 00:09:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
orders = spark.read.parquet("../../../data/insights/pre_insights/orders.parquet/")

In [4]:
orders.dtypes

[('merchant_abn', 'bigint'),
 ('merchant_name', 'string'),
 ('consumer_id', 'bigint'),
 ('order_datetime', 'date'),
 ('dollar_value', 'double')]

In [5]:
orders.show(truncate= False)

+------------+--------------------------------+-----------+--------------+------------+
|merchant_abn|merchant_name                   |consumer_id|order_datetime|dollar_value|
+------------+--------------------------------+-----------+--------------+------------+
|20985347699 |Semper Tellus PC                |1343547    |2021-08-21    |0.27        |
|45629217853 |Lacus Consulting                |1343547    |2021-08-14    |8.2         |
|71528203369 |Ipsum Primis Associates         |1343547    |2021-07-30    |10.07       |
|75900778714 |Laoreet Posuere Foundation      |1343547    |2021-08-20    |546.68      |
|66626020312 |In Magna Incorporated           |1343547    |2021-06-17    |57.54       |
|17324645993 |Eget Metus In Corporation       |1343547    |2021-05-29    |6.25        |
|13285958344 |Purus Gravida Sagittis Inc.     |1343547    |2021-05-02    |0.14        |
|99467302626 |Ac Ipsum Foundation             |1343547    |2021-06-21    |234.76      |
|41315101616 |Malesuada Fames Fo

In [6]:
orders_aggregated = orders.groupBy("merchant_abn").agg\
                        (f.count("order_datetime").alias("number_of_orders"),
                         f.avg("dollar_value").alias("average_cost_of_order"))

In [7]:
orders_aggregated.show(truncate=False)

+------------+----------------+---------------------+
|merchant_abn|number_of_orders|average_cost_of_order|
+------------+----------------+---------------------+
|83412691377 |11928           |34.9712743125419     |
|38700038932 |5944            |1344.3426917900406   |
|73256306726 |4361            |283.94453107085525   |
|41956465747 |244             |225.88536885245898   |
|19839532017 |614             |157.0                |
|35344855546 |1274            |89.1236813186813     |
|73841664453 |811             |85.53958076448829    |
|92202115241 |98              |330.3822448979592    |
|34440496342 |187             |89.50550802139037    |
|12516851436 |179             |155.86396648044695   |
|96946925998 |110             |1004.9574545454547   |
|15613631617 |1483            |303.77948078219816   |
|48214071373 |453             |303.3611479028698    |
|60654402457 |158             |85.27993670886076    |
|38986645707 |34              |1031.5729411764707   |
|78916025936 |52            

In [8]:
orders_aggregated.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/orders_agg.parquet")

In [9]:
spark.stop()